# Import Libraries and Clone Github

In [ ]:
# Import Libraries
import os
from google.colab import userdata
import google.generativeai as genai

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

# Import Data

In [ ]:
# Import all cleaned data files and merge into a df
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
df['Article Index'] = df.index
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date']= pd.to_datetime(df['Date']).dt.date
df

# Create Empty CSV for Results
### NOTE: DO NOT UNCOMMENT AND RUN CELL UNLESS YOU WANT TO DELETE EXISTING FILE

In [ ]:
# Create new df dropping text and adding new columns for the sentiment categories.  Put that df into a csv

# Sentiment Analysis

In [ ]:
# Set up a prompt

In [ ]:
# Set up a function.  It should query gemini, then parse the results directly into the csv.
# It also needs to continue by looking for the last empty entry and will need significant amounts of error handling

In [ ]:
# Run the function and print out results